<h1> Main.ipynb </h1>
<h2> 1. Import libraries and modules </h2>

In [1]:
import numpy as np
import pandas as pd

from src.Data_Retriever import DataRetriever as DatRet

from src.Optimization.Markowitz_PT import MarkowitzPT as MPT

from src.Optimization.Environment import PortfolioEnvironment as PorEnv
from src.Optimization.RLModelCompilation import RL_Model as RLM
# from src.Optimization.NeuralNet import CustomCNNExtractor 

from src.Result.Menchero_OGA import MencheroOGA as MOGA
from src.Result.IndPortResults import GenerateResult as GR
from src.Result.OverviewResults import ResultConveyor as RC

import time

<h2> 2. Define operating variables </h2>

In [2]:
trading_n = 400
history_usage = 504
n_sectors = 6
n_stocks_per_sector = 4

# For RL algorithm
history_usage_RL = 30
rolling_reward_window = 15

<h2> 3. Define stock dataframe </h2>

In [3]:
ticker_df =  pd.DataFrame()
ticker_df["Petroleum"] = ["EQNR.OL", "AKRBP.OL", "SUBC.OL", "BWO.OL",]
ticker_df["Seafood (food)"] = ["ORK.OL", "MOWI.OL", "SALM.OL", "LSG.OL"]
ticker_df["Materials"] = ["NHY.OL", "YAR.OL", "RECSI.OL", "BRG.OL"]  #del this
ticker_df["Technologies"] = ["TEL.OL", "NOD.OL", "ATEA.OL", "BOUV.OL"]
ticker_df["Financial"] = ["STB.OL", "DNB.OL", "GJF.OL", "AKER.OL"]
ticker_df["Shipping"] = ["WAWI.OL", "SNI.OL", "BELCO.OL", "ODF.OL"]
ticker_df

,Petroleum,Seafood (food),Materials,Technologies,Financial,Shipping
0,EQNR.OL,ORK.OL,NHY.OL,TEL.OL,STB.OL,WAWI.OL
1,AKRBP.OL,MOWI.OL,YAR.OL,NOD.OL,DNB.OL,SNI.OL
2,SUBC.OL,SALM.OL,RECSI.OL,ATEA.OL,GJF.OL,BELCO.OL
3,BWO.OL,LSG.OL,BRG.OL,BOUV.OL,AKER.OL,ODF.OL


<h2> 4. Define ESG array </h2>

In [4]:
esg_scores = np.array([36.6, 35.3, 17.9, 18, 
                18, 21.2, 18.7, 29.2, 
                15.7, 25.6, 25.6, 18.4, # Del this
                19.8, 13.8, 18.1, 19, 
                17.2, 14, 17.2, 19.5, 
                19.7, 21.2, 26.8, 19.3])

<h2> 5.-7. (Non-necessary for reproducability) </h2>
<h3>Data retrieval</h3>
<h3>Benchmark optimization (MPT)</h3>
<h3>RL optimization x4</h3>
<br>
All of the data has been generated an stored in csvs. Therefore, I can comment out the code lines below.

In [5]:
# Retrieve data from yf API: y-m-d
# data = DatRet(ticker_df, "2013-01-01", "2024-12-31", history_usage_RL=history_usage_RL)
# # In function below, set log=True to check for data availability
# data.retrieve_data()

In [6]:
# Generate benchmark weights thorugh MPT using Sharpe ratio
# # benchmark = MPT(history_usage, trading_n)
# IMPORTANT: In order to see  the effect of the weights, algo exclude last observation from optimization
# # benchmark.frequency_optimizing()

In [7]:
# objectives = ["Return", "Sharpe", "Sortino", "Sterling", "Return", "Sharpe", "Sortino", "Sterling"]
# esg_compliancy = [True, True, True, True, False, False, False, False]
# # objectives = ["Sharpe"]
# # esg_compliancy = [True]

# for i in range(len(objectives)):
#     reinforcement = RLM(esg_scores, 
#                         objective=objectives[i],
#                         history_usage=history_usage_RL,
#                         rolling_reward_window=rolling_reward_window,
#                         total_timesteps=200,
#                         esg_compliancy=esg_compliancy[i],
#                         )
#     reinforcement.train_model()
#     reinforcement.test_model()

<h2> 8. Attribution analysis </h2>

In [8]:
# paths = ["Return_esg_True", "Sharpe_esg_True",
#          "Sortino_esg_True","Sterling_esg_True",
#          "Return_esg_False", "Sharpe_esg_False",
#          "Sortino_esg_False","Sterling_esg_False",]

# analysis_list = []
# for i in range(len(paths)):
#     att_anal = GR(paths[i],
#             n_sectors, n_stocks_per_sector,
#             trading_n,
#             esg_scores, 
#             ticker_df.columns)
#     att_anal.friple_frequency_analysis()
#     analysis_list.append(att_anal)

<h2> 9. Overview  Result Analysis </h2>

In [9]:
# theta = RC(analysis_list, trading_n)
# theta.convey_results()

<h1> 10. Ad-hoc </h1>

In [10]:


def make_env():
    env = PorEnv(
        history_usage=history_usage,
        rolling_reward_window=rolling_reward_window,
        esg_data=esg_scores,
        objective="Sharpe",
        esg_compliancy=True
    )
    return env

train_env = make_env()
test_env = make_env()

In [11]:
from stable_baselines3 import SAC


In [3355]:
model = SAC(
    policy="MlpPolicy",
    env=train_env,
    learning_rate=3e-4,
    buffer_size=100_000,
    batch_size=256,
    ent_coef="auto",
    target_entropy="auto",
    verbose=1,
).learn(total_timesteps=400)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Step: 1 , Reward:  0.0 weights:  [0.9552399  0.00384995 0.14403522 0.46327478 0.8451614  0.17907059
 0.21369854 0.34961334 0.2280615  0.0921357  0.07911462 0.5543798
 0.7159076  0.32888183 0.46296707 0.735986   0.39601135 0.44419986
 0.69159526 0.39248922 0.4227418  0.8968225  0.9965699  0.6917461 ]
Step: 2 , Reward:  0.0 weights:  [0.20091751 0.8797969  0.11750087 0.25350788 0.76681113 0.9009633
 0.84025526 0.9859346  0.34709087 0.14263272 0.85805655 0.465418
 0.4252402  0.46136168 0.7400216  0.96971583 0.08947447 0.34694868
 0.87461495 0.2284618  0.2691505  0.01231354 0.61148566 0.69283307]
Step: 3 , Reward:  1.4120364347152459 weights:  [0.6753735  0.26454827 0.25375912 0.03325877 0.21425182 0.4976236
 0.5864686  0.2488625  0.76298994 0.5118893  0.15382281 0.8724906
 0.90986466 0.9377899  0.50954264 0.5818036  0.8506481  0.33702755
 0.55896086 0.56808215 0.972205   0.09838361 0.76098806 0.1

In [3356]:
obs, _ = train_env.reset()
actions, _ = model.predict(obs, deterministic=True)
print(actions)

[0.49189955 0.5135144  0.4625722  0.48429513 0.5006337  0.49360344
 0.5392966  0.50351894 0.54371846 0.5640096  0.4732304  0.5113648
 0.5395701  0.5172388  0.46664128 0.5218054  0.5012927  0.55177593
 0.4724523  0.45760816 0.40061602 0.4478844  0.47480032 0.5205927 ]


In [3357]:
norm = actions / np.sum(actions)
print(norm)

[0.04114959 0.04295777 0.03869623 0.04051344 0.04188024 0.04129213
 0.04511457 0.0421216  0.04548447 0.04718192 0.03958783 0.04277795
 0.04513744 0.04326933 0.03903662 0.04365135 0.04193537 0.04615851
 0.03952274 0.03828096 0.03351332 0.03746753 0.03971916 0.0435499 ]


In [3358]:
train_env.step(norm)

Step: 1 , Reward:  0.0 weights:  [0.04114959 0.04295777 0.03869623 0.04051344 0.04188024 0.04129213
 0.04511457 0.0421216  0.04548447 0.04718192 0.03958783 0.04277795
 0.04513744 0.04326933 0.03903662 0.04365135 0.04193537 0.04615851
 0.03952274 0.03828096 0.03351332 0.03746753 0.03971916 0.0435499 ]


(array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        , -0.14875537,
        -0.20019796, -0.15338792, -0.13869327, -0.17764835, -0.16650148,
        -0.13914135, -0.2101108 , -0.12466612, -0.1701478 , -0.22064877,
        -0.14484504, -0.09933816, -0.26717907, -0.08440344, -0.20676866,
        -0.24074557, -0.18713956, -0.06541503, -0.18698472, -0.25438488,
        -0.40900496, -0.13383524, -0.20548418, -2.1626632 , -2.5300395 ,
        -2.7232282 , -2.9154546 , -2.2206202 , -2.1776798 , -2.0686636 ,
        -2.7118142 , -2.9217448 , -2.8446975 , -1.7116281 , -2.7521422 ,
        -1.915301  , -2.702168  , -2.2993932 , -2.6029973 , -2.5585344 ,
        -2.1853278 , -1.8066901 , -2.9620786 , -3.3

In [5260]:
actions, _ = model.predict(obs, deterministic=False)
print(actions)

[0.97150946 0.56987303 0.0306825  0.11110437 0.52139443 0.75964534
 0.13051689 0.66843003 0.44473985 0.75057244 0.7459935  0.91527474
 0.39351064 0.6792741  0.09107763 0.8653203  0.94605327 0.12145942
 0.56719285 0.10465443 0.6602806  0.5610862  0.2229901  0.43685794]


In [5261]:
norm = actions / np.sum(actions)
print(norm)

[0.07918089 0.04644633 0.00250071 0.00905533 0.04249519 0.06191334
 0.01063751 0.05447902 0.03624761 0.06117387 0.06080067 0.0745976
 0.03207228 0.05536284 0.0074231  0.07052616 0.07710613 0.0098993
 0.04622789 0.00852965 0.05381482 0.04573018 0.01817435 0.03560521]


In [5262]:
train_env.step(norm)

Step: 227 , Reward:  0.17515288000831494 weights:  [0.07918089 0.04644633 0.00250071 0.00905533 0.04249519 0.06191334
 0.01063751 0.05447902 0.03624761 0.06117387 0.06080067 0.0745976
 0.03207228 0.05536284 0.0074231  0.07052616 0.07710613 0.0098993
 0.04622789 0.00852965 0.05381482 0.04573018 0.01817435 0.03560521]


(array([-3.0839932e-03, -2.0031747e-03,  7.4677187e-04,  0.0000000e+00,
         7.5269174e-03,  1.3071865e-02,  2.4900958e-02, -8.3540916e-04,
        -2.4725286e-02,  1.5789434e-02,  1.8151831e-02,  3.7735850e-03,
         1.5576324e-02, -2.1097047e-02,  3.2921780e-03,  8.4745763e-03,
         1.7888002e-02,  0.0000000e+00, -7.9364432e-03, -1.6077170e-02,
        -2.1505391e-02,  0.0000000e+00, -1.1111111e-02, -2.9508185e-02,
        -4.5030439e-01, -8.0497861e-01, -1.2996925e-01, -8.3653027e-01,
         4.9206063e-01, -2.6468271e-02,  6.2180907e-01, -6.4603347e-01,
        -6.6155171e-01,  7.3009521e-01, -7.9411703e-01,  3.0352041e-01,
        -9.3837790e-03, -8.3645827e-01,  2.1069255e+00,  1.6749818e-02,
        -7.0310152e-01, -1.1653206e+00,  1.4086002e+00, -1.6458750e+00,
         9.1809922e-01,  1.8464211e+00, -6.9617577e-02,  3.6842531e-01,
        -1.1970291e+00, -7.9331511e-01, -1.2791957e+00, -1.0912576e+00,
        -4.8952281e-01, -6.0211456e-01, -5.6244129e-01, -6.97541

<!--  -->

In [5263]:
import yfinance as yf

In [5264]:
# class NewData:

#     def __init__(self,
#                  start_date:str,
#                  end_date:str,
#                  ticker_df: pd.DataFrame):
#         self.start_date = start_date
#         self.end_date = end_date
#         self.ticker_df = ticker_df
#         self.tickers = np.array(ticker_df.values.tolist()).flatten()
#         self.master_date_range = pd.Series(np.repeat(pd.date_range(start=start_date, end=end_date, freq="B"), 2), name="Date")

#         self.return_df: pd.DataFrame = None
#         self.volume_df: pd.DataFrame = None
#         self.rolling_vol_df: pd.DataFrame = None
#         self.rolling_ret_df: pd.DataFrame = None
    

#     def z_score(self, data: pd.DataFrame):

#         score_df = (data-data.mean()) / data.std()
#         return score_df

#     def download_data(self):
#         ret_df = pd.DataFrame()
#         vol_df = pd.DataFrame()

#         for i in range(0,len(self.tickers),1):
#             data = yf.download(self.tickers[i], start=self.start_date, end=self.end_date)
#             open_merge_df = pd.merge(self.master_date_range, data["Open"].copy(), on="Date", how="left")
#             open_merge_df.loc[1::2, self.tickers[i]] = np.nan
#             close_merge_df = pd.merge(self.master_date_range, data["Close"].copy(), on="Date", how="left")
#             close_merge_df.loc[::2, self.tickers[i]] = np.nan
#             open_merge_df.loc[1::2, self.tickers[i]] = close_merge_df
#             individual_df = open_merge_df.interpolate().ffill().bfill()
#             vol_series = pd.merge(self.master_date_range, data["Volume"], on="Date", how="left").interpolate().ffill().bfill()

#             vol_df[self.tickers[i]] = vol_series[self.tickers[i]]
#             ret_df[self.tickers[i]] = individual_df[self.tickers[i]]
        
#         self.return_df = ret_df.pct_change(1).bfill()
#         self.volume_df = self.z_score(vol_df)
#         self.rolling_ret_df = self.z_score(self.return_df.rolling(42).sum().fillna(0))
#         self.rolling_vol_df = self.z_score(self.return_df.rolling(42).std().fillna(0))

#         self.return_df.to_csv("Data/Input/StockReturns.csv", index=False)
#         self.volume_df.to_csv("Data/Input/Volume.csv", index=False)
#         self.rolling_ret_df.to_csv("Data/Input/RollingRet.csv", index=False)
#         self.rolling_vol_df.to_csv("Data/Input/RollingVol.csv", index=False)

In [5265]:
# new_data = NewData("2013-01-01", "2024-12-31", ticker_df)
# new_data.download_data()

In [5266]:
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize, VecCheckNan


In [5267]:
env = PorEnv(history_usage=80, rolling_reward_window=80, esg_data=esg_scores, objective="Sharpe",  esg_compliancy=True)

In [5268]:
myenv = DummyVecEnv([lambda: env])

# Verify shapes
print("Observation space shape:", env.observation_space.shape)
sample_obs, _ = env.reset()
print("Actual obs shape:", sample_obs.shape)  # Should match

Observation space shape: (120,)
Actual obs shape: (120,)
